In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

2025-09-03 05:55:48.956314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756878949.172041      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756878949.233837      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


CIFAR 100 dataset has 20 superclasses each containing 5 subclasses.

So now I will be training 20 different models on each of the superclasses and then train the gating model for Mixture Of Experts.

So now for the 20 different models, I will be creating seperate training data and labels.

In [29]:
#This dictionary maps 100 fine grained classes to 20 coearse grained superclasses.

fine_to_coarse_labels = {}

for i in range(0,100):
    n = i // 5
    fine_to_coarse_labels[i] = n

    
superclass_names = [
    "aquatic mammals", "fish", "flowers", "food containers", "fruit and vegetables",
    "household electrical devices", "household furniture", "insects", "large carnivores",
    "large man-made outdoor things", "large natural outdoor scenes",
    "large omnivores and herbivores", "medium-sized mammals", "non-insect invertebrates",
    "people", "reptiles", "small mammals", "trees", "vehicles 1", "vehicles 2"
]

def get_superclass_data():
    (x_train_fine, y_train_fine), (x_test_fine, y_test_fine) = tf.keras.datasets.cifar100.load_data(label_mode='fine')
    superclass_data = {}

    for name in superclass_names:
        superclass_data[name] = {'images':[], 'labels':[]}

    for i in range(len(x_train_fine)):
        fine_label = y_train_fine[i][0]
        coarse_label = fine_to_coarse_labels[fine_label]
        superclass_name = superclass_names[coarse_label]

        superclass_data[superclass_name]['images'].append(x_train_fine[i])
        superclass_data[superclass_name]['labels'].append(fine_label)

    for name in superclass_names:
        images = np.array(superclass_data[name]['images'])
        labels = np.array(superclass_data[name]['labels'])
        superclass_data[name] = (images,labels)
    
    return superclass_data

In [30]:
superclass_data = get_superclass_data()

In [31]:
all_data = list(superclass_data.values())

In [32]:
all_images = []
all_labels = []

for tup in all_data:
    tup_l = list(tup)
    all_images.append(tup_l[0])
    all_labels.append(tup_l[1])
    

Now building and training 20 different models each for a different superclass.

In [33]:
def build_model(num_classes):
    model = Sequential([
       # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        # Flatten the feature maps to feed into the dense layers
        Flatten(),
        # Fully Connected Layers
        Dense(128, activation='relu'),
        # Output layer with 'num_classes' neurons for classification
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

    return model

def train_single_superclass_model(x_train_superclass,y_train_superclass,superclass_name):
    y_train_flat = y_train_superclass.flatten()
    unique_labels = np.unique(y_train_flat)
    num_classes = len(unique_labels)
    label_map = {label: i for i, label in enumerate(unique_labels)}
    y_train_superclass_list = y_train_superclass.tolist()
    y_train_mapped = np.array([label_map[label] for label in y_train_flat])

    print(f"\n--- Building and training model for superclass: {superclass_name} ---")
    print(f"Number of unique fine classes: {num_classes}")

    model = build_model(num_classes)
    model.fit(x_train_superclass, y_train_mapped, epochs = 5,batch_size = 32, verbose = 1)

    print(f"Training for {superclass_name} complete.")
    return model

In [34]:
aq_m_x_train = all_images[0]
fish_x_train = all_images[1]
flwr_x_train = all_images[2]
fooc_x_train = all_images[3]
fav_x_train = all_images[4]
hed_x_train = all_images[5]
hf_x_train = all_images[6]
ins_x_train = all_images[7]
lc_x_train = all_images[8]
lmot_x_train = all_images[9]
lnos_x_train = all_images[10]
loah_x_train = all_images[11]
msm_x_train = all_images[12]
nii_x_train = all_images[13]
peo_x_train = all_images[14]
rep_x_train = all_images[15]
sm_x_train = all_images[16]
trees_x_train = all_images[17]
v1_x_train = all_images[18]
v2_x_train = all_images[19]


aq_m_y_train = all_labels[0].reshape(-1,1)
fish_y_train = all_labels[1].reshape(-1,1)
flwr_y_train = all_labels[2].reshape(-1,1)
fooc_y_train = all_labels[3].reshape(-1,1)
fav_y_train = all_labels[4].reshape(-1,1)
hed_y_train = all_labels[5].reshape(-1,1)
hf_y_train = all_labels[6].reshape(-1,1)
ins_y_train = all_labels[7].reshape(-1,1)
lc_y_train = all_labels[8].reshape(-1,1)
lmot_y_train = all_labels[9].reshape(-1,1)
lnos_y_train = all_labels[10].reshape(-1,1)
loah_y_train = all_labels[11].reshape(-1,1)
msm_y_train = all_labels[12].reshape(-1,1)
nii_y_train = all_labels[13].reshape(-1,1)
peo_y_train = all_labels[14].reshape(-1,1)
rep_y_train = all_labels[15].reshape(-1,1)
sm_y_train = all_labels[16].reshape(-1,1)
trees_y_train = all_labels[17].reshape(-1,1)
v1_y_train = all_labels[18].reshape(-1,1)
v2_y_train = all_labels[19].reshape(-1,1)

In [35]:
aq_model = train_single_superclass_model(aq_m_x_train,aq_m_y_train,superclass_names[0])


--- Building and training model for superclass: aquatic mammals ---
Number of unique fine classes: 5
Epoch 1/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.3024 - loss: 17.3028
Epoch 2/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.5739 - loss: 1.0589
Epoch 3/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.6585 - loss: 0.8555
Epoch 4/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.7440 - loss: 0.6783
Epoch 5/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.8086 - loss: 0.5275
Training for aquatic mammals complete.


In [23]:
fine_coarse_mapping

{1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 2,
 12: 2,
 13: 2,
 14: 2,
 15: 2,
 16: 3,
 17: 3,
 18: 3,
 19: 3,
 20: 3,
 21: 4,
 22: 4,
 23: 4,
 24: 4,
 25: 4,
 26: 5,
 27: 5,
 28: 5,
 29: 5,
 30: 5,
 31: 6,
 32: 6,
 33: 6,
 34: 6,
 35: 6,
 36: 7,
 37: 7,
 38: 7,
 39: 7,
 40: 7,
 41: 8,
 42: 8,
 43: 8,
 44: 8,
 45: 8,
 46: 9,
 47: 9,
 48: 9,
 49: 9,
 50: 9,
 51: 10,
 52: 10,
 53: 10,
 54: 10,
 55: 10,
 56: 11,
 57: 11,
 58: 11,
 59: 11,
 60: 11,
 61: 12,
 62: 12,
 63: 12,
 64: 12,
 65: 12,
 66: 13,
 67: 13,
 68: 13,
 69: 13,
 70: 13,
 71: 14,
 72: 14,
 73: 14,
 74: 14,
 75: 14,
 76: 15,
 77: 15,
 78: 15,
 79: 15,
 80: 15,
 81: 16,
 82: 16,
 83: 16,
 84: 16,
 85: 16,
 86: 17,
 87: 17,
 88: 17,
 89: 17,
 90: 17,
 91: 18,
 92: 18,
 93: 18,
 94: 18,
 95: 18,
 96: 19,
 97: 19,
 98: 19,
 99: 19,
 100: 19}